In [2]:
data_path = '../tensor_data/'

In [43]:
import numpy as np
from jax import vmap
import jax.numpy as jnp
import os
import pennylane as qml
from typing import Tuple

In [87]:
class mpsANNNI:
    def __init__(self, folder : str = '../tensor_data/'):
        self.path = folder

        # Check if folder exists
        try: 
            files_all = np.array(os.listdir(folder))
            self.files_shape = files_all[np.char.startswith(files_all,'shapes_sites')]
            self.files_tens  = files_all[np.char.startswith(files_all,'tensor_sites')]
        except:
            raise TypeError(f'Folder {folder} not found')
        
        def get_info(file_string : str) -> Tuple[int, float, float]:
            """
            1. Split big string into array of string
            'shapes_sites_ANNNI_L_{L}_h_{h}_kappa_{k}'
                into
            ['shapes', 'sites', 'ANNNI', 'L', '{L}','h', '{h}', 'kappa', '{k}']
              0         1        2        3    4^    5    6^     7        8^
            2. Take the element 4, 6 and 8 
            """
            split_str = file_string.split('_')

            # return respectively:
            # L, h, k, precision on h, precision on k
            return int(split_str[4]), float(split_str[6]), float(split_str[8]), len(split_str[6].split('.')[1]), len(split_str[8].split('.')[1])
        
        # Check if files are okay
        Ls_shape, hs_shape, ks_shape, hs_shape_prec, ks_shape_prec = [], [], [], [], []
        Ls_tens,  hs_tens,  ks_tens,  hs_tens_prec,  ks_tens_prec  = [], [], [], [], []
        for file in self.files_shape:
            L, h, k, hprec, kprec = get_info(file)
            Ls_shape.append(L)
            hs_shape.append(h)
            ks_shape.append(k)
            hs_shape_prec.append(hprec)
            ks_shape_prec.append(kprec)
        for file in self.files_tens:
            L, h, k, hprec, kprec = get_info(file)
            Ls_tens.append(L)
            hs_tens.append(h)
            ks_tens.append(k)
            hs_tens_prec.append(hprec)
            ks_tens_prec.append(kprec)

        # Check on L
        if len(np.unique(Ls_shape)) > 1 or len(np.unique(Ls_tens)) > 1:
            raise ValueError(f'L has multiple values')
        elif Ls_shape[0] != Ls_tens[0]:
            raise ValueError(f'L has inconsistent values')
        # otherwise L is okay:
        self.L = Ls_shape[0]

        # Check on h and k
        #  None for now
        self.hs = np.unique(hs_shape)
        self.ks = np.unique(ks_shape)

        # Check on precisions
        if len(np.unique(hs_shape_prec + hs_tens_prec)) > 1 or len(np.unique(ks_shape_prec + ks_tens_prec)) > 1: 
            raise ValueError('Inconsistent precisions in files')
        self.h_prec = hs_shape_prec[0]
        self.k_prec = ks_shape_prec[0]

        # Format of the file names:
        # shape_file  : shape_sites_ANNNI_L_{N}_h_{h}_kappa_{k}
        self.shape_str  = lambda h, k : folder+f'shapes_sites_ANNNI_L_{self.L}_h_{h:.{self.h_prec}f}_kappa_{k:.{self.k_prec}f}'
        # tensor_file : shape_sites_ANNNI_L_{N}_h_{h}_kappa_{k}
        self.tensor_str = lambda h, k : folder+f'tensor_sites_ANNNI_L_{self.L}_h_{h:.{self.h_prec}f}_kappa_{k:.{self.k_prec}f}'

    def get_H(self, h, k):
        """
        Set up Hamiltonian
        """

        # Interaction of spins with magnetic field
        H = - h * qml.PauliZ(0)
        for i in range(1, self.L):
            H = H - h * qml.PauliZ(i)

        # Interaction between spins (neighbouring):
        for i in range(0, self.L - 1):
            H = H + (-1) * (qml.PauliX(i) @ qml.PauliX(i + 1))

        # Interaction between spins (next-neighbouring):
        for i in range(0, self.L - 2):
            H = H + (-1) * k * (qml.PauliX(i) @ qml.PauliX(i + 2))

        return H       
        



In [88]:
mps = mpsANNNI() # load the folder

In [95]:
print("Here are all the information obtained from the folder:")
print(f' L = {mps.L}\n\n hs = {mps.hs:}\n hs (precision) = {mps.h_prec}\n\n ks = {mps.ks}\n ks (precision) = {mps.k_prec}')

Here are all the information obtained from the folder:
 L = 12

 hs = [0.   0.1  0.11 0.2  0.21 0.3  0.32 0.4  0.42 0.5  0.53 0.6  0.63 0.7
 0.74 0.8  0.84 0.9  0.95 1.   1.05 1.1  1.16 1.2  1.26 1.3  1.37 1.4
 1.47 1.58 1.68 1.79 1.89 2.  ]
 hs (precision) = 2

 ks = [0.   0.05 0.1  0.11 0.15 0.16 0.2  0.21 0.25 0.26 0.3  0.32 0.35 0.37
 0.4  0.42 0.45 0.47 0.5  0.53 0.55 0.58 0.6  0.63 0.65 0.68 0.7  0.74
 0.75 0.79 0.8  0.84 0.85 0.89 0.9  0.95 1.  ]
 ks (precision) = 2


In [102]:
# I can also easily load the files
print(f'SHAPE:\n {np.loadtxt(mps.tensor_str(0,0))}\n')
print(f'TENS:\n {np.loadtxt(mps.shape_str(0,0))}')


SHAPE:
 [-0.70710678  0.70710678 -0.70710678  0.70710678 -0.70710678  0.70710678
 -0.70710678  0.70710678 -0.70710678  0.70710678  0.70710678 -0.70710678
 -0.70710678  0.70710678  0.70710678 -0.70710678  0.70710678 -0.70710678
 -0.70710678  0.70710678 -0.70710678  0.70710678 -0.70710678  0.70710678]

TENS:
 [[1. 2. 1.]
 [1. 2. 1.]
 [1. 2. 1.]
 [1. 2. 1.]
 [1. 2. 1.]
 [1. 2. 1.]
 [1. 2. 1.]
 [1. 2. 1.]
 [1. 2. 1.]
 [1. 2. 1.]
 [1. 2. 1.]
 [1. 2. 1.]]
